<a href="https://colab.research.google.com/github/blkbamboo74/DS-Unit-2-Linear-Models/blob/master/Dennis___Batiste__LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`). 

Use a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.

- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Fit a ridge regression model with multiple features.
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
# ('BUILDING_CLASS_CATEGORY' == '01 ONE FAMILY DWELLINGS')

df = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']
df = df.drop('EASE-MENT', axis = 1)
df = df.drop('APARTMENT_NUMBER', axis = 1)

In [30]:
# Do train/test split
# Use data from January - March 2019 to train
# Use data from April 2019 to test
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format=True)
cutoff = pd.to_datetime('2019-04-01')
train = df[df.SALE_DATE < cutoff]
test  = df[df.SALE_DATE >= cutoff]
print(train.shape, test.shape)


(4094, 19) (967, 19)


In [31]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
7,2,OTHER,01 ONE FAMILY DWELLINGS,1,4090,37,A1,1193 SACKET AVENUE,10461.0,1.0,0.0,1.0,"3,404",1328.0,1925.0,1,A1,0,2019-01-01
8,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,18,A5,1215 VAN NEST AVENUE,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,2019-01-01
9,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,20,A5,1211 VAN NEST AVENUE,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,2019-01-01
42,3,OTHER,01 ONE FAMILY DWELLINGS,1,6809,54,A1,2601 AVENUE R,11229.0,1.0,0.0,1.0,"3,333",1262.0,1925.0,1,A1,0,2019-01-01
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,2019-01-01


In [32]:
df.shape

(5061, 19)

In [33]:
df.dtypes

BOROUGH                                   object
NEIGHBORHOOD                              object
BUILDING_CLASS_CATEGORY                   object
TAX_CLASS_AT_PRESENT                      object
BLOCK                                      int64
LOT                                        int64
BUILDING_CLASS_AT_PRESENT                 object
ADDRESS                                   object
ZIP_CODE                                 float64
RESIDENTIAL_UNITS                        float64
COMMERCIAL_UNITS                         float64
TOTAL_UNITS                              float64
LAND_SQUARE_FEET                          object
GROSS_SQUARE_FEET                        float64
YEAR_BUILT                               float64
TAX_CLASS_AT_TIME_OF_SALE                  int64
BUILDING_CLASS_AT_TIME_OF_SALE            object
SALE_PRICE                                 int64
SALE_DATE                         datetime64[ns]
dtype: object

Split the test data and properly test the model

Find columsn that are numeric and those that aren't

In [0]:
target = 'SALE_PRICE'
high_cardinality = ['NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY'
,'TAX_CLASS_AT_PRESENT','BUILDING_CLASS_AT_PRESENT','ADDRESS'
,'LAND_SQUARE_FEET','BUILDING_CLASS_AT_TIME_OF_SALE',
'SALE_DATE']
features = train.columns.drop([target] + high_cardinality)

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [10]:
X_train.head()

,BOROUGH,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
0,1,716,1246,NaN,10011.0,1.0,0.0,1.0,1979.0,2007.0,2
1,1,812,68,NaN,10018.0,0.0,6.0,6.0,15435.0,1920.0,4
2,1,839,69,NaN,10018.0,0.0,7.0,7.0,11332.0,1930.0,4
3,1,592,1041,NaN,10014.0,1.0,0.0,1.0,500.0,0.0,2
4,1,1379,1402,NaN,10065.0,1.0,0.0,1.0,6406.0,0.0,2


In [36]:
train.select_dtypes(include='number').describe().T

,count,mean,std,min,25%,50%,75%,max
BLOCK,4094.0,6665.769419,3911.046025,17.0,3811.00,6026.5,9501.75,16350.0
LOT,4094.0,72.974841,152.843450,1.0,20.25,42.0,69.00,2720.0
ZIP_CODE,4094.0,11012.498534,516.706900,0.0,10453.00,11234.0,11413.00,11697.0
RESIDENTIAL_UNITS,4094.0,0.991451,0.092076,0.0,1.00,1.0,1.00,1.0
COMMERCIAL_UNITS,4094.0,0.024670,0.159792,0.0,0.00,0.0,0.00,2.0
TOTAL_UNITS,4094.0,1.016121,0.185562,0.0,1.00,1.0,1.00,3.0
GROSS_SQUARE_FEET,4094.0,1571.044944,754.804640,0.0,1170.00,1408.0,1785.75,12618.0
YEAR_BUILT,4094.0,1943.012702,40.361466,0.0,1925.00,1940.0,1955.00,2018.0
TAX_CLASS_AT_TIME_OF_SALE,4094.0,1.000000,0.000000,1.0,1.00,1.0,1.00,1.0
SALE_PRICE,4094.0,457941.816317,888519.851507,0.0,0.00,414693.5,630000.00,30139360.0


Now for the non numeric

In [37]:
train.select_dtypes(exclude='number').describe().T

,count,unique,top,freq,first,last
BOROUGH,4094,5,4,1993,NaT,NaT
NEIGHBORHOOD,4094,9,OTHER,3826,NaT,NaT
BUILDING_CLASS_CATEGORY,4094,1,01 ONE FAMILY DWELLINGS,4094,NaT,NaT
TAX_CLASS_AT_PRESENT,4094,3,1,4058,NaT,NaT
BUILDING_CLASS_AT_PRESENT,4094,15,A1,1542,NaT,NaT
ADDRESS,4094,4059,277 NICHOLS AVENUE,3,NaT,NaT
LAND_SQUARE_FEET,4094,1280,"4,000",412,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,4094,12,A1,1543,NaT,NaT
SALE_DATE,4094,90,2019-03-29 00:00:00,96,2019-01-01,2019-03-31


We are going to use Borough for our test.   

Shot X_train before encoding

In [38]:
print(X_train.shape)
X_train.head()

(4094, 10)


,BOROUGH,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
7,2,4090,37,10461.0,1.0,0.0,1.0,1328.0,1925.0,1
8,2,4120,18,10461.0,1.0,0.0,1.0,1728.0,1935.0,1
9,2,4120,20,10461.0,1.0,0.0,1.0,1728.0,1935.0,1
42,3,6809,54,11229.0,1.0,0.0,1.0,1262.0,1925.0,1
44,3,5495,801,11230.0,1.0,0.0,1.0,1325.0,1930.0,1


In [0]:
import category_encoders as ce 

encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [40]:
print (X_train.shape)
X_train.head()

(4094, 14)


,BOROUGH_2,BOROUGH_3,BOROUGH_4,BOROUGH_5,BOROUGH_1,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE
7,1,0,0,0,0,4090,37,10461.0,1.0,0.0,1.0,1328.0,1925.0,1
8,1,0,0,0,0,4120,18,10461.0,1.0,0.0,1.0,1728.0,1935.0,1
9,1,0,0,0,0,4120,20,10461.0,1.0,0.0,1.0,1728.0,1935.0,1
42,0,1,0,0,0,6809,54,11229.0,1.0,0.0,1.0,1262.0,1925.0,1
44,0,1,0,0,0,5495,801,11230.0,1.0,0.0,1.0,1325.0,1930.0,1


We get the five seperate catagoires for borough

## SelectKBest

In [41]:
pd.isnull(df).sum() > 0

BOROUGH                           False
NEIGHBORHOOD                      False
BUILDING_CLASS_CATEGORY           False
TAX_CLASS_AT_PRESENT              False
BLOCK                             False
LOT                               False
BUILDING_CLASS_AT_PRESENT         False
ADDRESS                           False
ZIP_CODE                          False
RESIDENTIAL_UNITS                 False
COMMERCIAL_UNITS                  False
TOTAL_UNITS                       False
LAND_SQUARE_FEET                  False
GROSS_SQUARE_FEET                 False
YEAR_BUILT                        False
TAX_CLASS_AT_TIME_OF_SALE         False
BUILDING_CLASS_AT_TIME_OF_SALE    False
SALE_PRICE                        False
SALE_DATE                         False
dtype: bool

In [42]:
pd.isnull(df).sum()

BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING_CLASS_CATEGORY           0
TAX_CLASS_AT_PRESENT              0
BLOCK                             0
LOT                               0
BUILDING_CLASS_AT_PRESENT         0
ADDRESS                           0
ZIP_CODE                          0
RESIDENTIAL_UNITS                 0
COMMERCIAL_UNITS                  0
TOTAL_UNITS                       0
LAND_SQUARE_FEET                  0
GROSS_SQUARE_FEET                 0
YEAR_BUILT                        0
TAX_CLASS_AT_TIME_OF_SALE         0
BUILDING_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                        0
SALE_DATE                         0
dtype: int64

In [43]:
df.shape

(5061, 19)

In [47]:
from sklearn.feature_selection import f_regression, SelectKBest

selector = SelectKBest(score_func=f_regression, k='all')
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)
X_train_selected.shape

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


(4094, 14)

In [0]:
# - [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
from sklearn import preprocessing

In [0]:
# - [X] Fit a ridge regression model with multiple features.
from sklearn.linear_model import Ridge

model = Ridge(alpha = 15)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [50]:
# - [X] Get mean absolute error for the test set.
                     import numpy as np
from sklearn.metrics import mean_absolute_error


mean_absolute_error(y_test, y_pred)

390363.0498138835

Over all I think that I made this way harder on myself.  I think I could have done this in a way that is way less code, but focused on funciton. 